# Doc2Vecの初期動作確認

こちらを参考にしました。


- Doc2Vec Tutorial on the Lee Dataset

　　https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb

仮決めのパラメータにて、質問に対する近似度が戻るところまで確認できました。

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

## (2) Doc2Vecの動作確認

### (2-1) コーパス生成

既存実装どおり、Nlangクラスを使い、コーパス（単語が半角スペースで区切られた文字列）を生成します。

In [2]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 9  # bot_id = 9はセプテーニ
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}

learning_parameter = LearningParameter(attr)

In [3]:
_datasource = Datasource(type='csv')
learning_training_messages = _datasource.learning_training_messages(_bot_id)
questions = np.array(learning_training_messages['question'])
answer_ids = np.array(learning_training_messages['answer_id'])

2017/05/16 PM 01:50:11 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/16 PM 01:50:11 ['./fixtures/question_answers/toyotsu_human.csv']


In [4]:
from learning.core.nlang import Nlang

_sentences = np.array(questions)
_separated_sentences = Nlang.batch_split(_sentences)

### (2-2) コーパスにタグ付け

models.doc2vecの仕様に従います。

In [5]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [6]:
def doc_to_sentence(sentences, name):
    words = sentences.split(' ')
    return TaggedDocument(words=words, tags=[name])

def corpus_to_sentences(separated_sentences, answer_ids):
    for idx, (doc, name) in enumerate(zip(separated_sentences, answer_ids)):
        yield doc_to_sentence(doc, name)

### (2-3) 学習処理／モデルのシリアライズ

In [7]:
sentences = corpus_to_sentences(_separated_sentences, answer_ids)

In [8]:
sentence_list = list(sentences)

In [9]:
model = Doc2Vec(size=500, min_count=1, iter=200)

In [10]:
model.build_vocab(sentence_list)

In [11]:
model.train(sentence_list)

13682822

In [12]:
model_path = 'prototype/better_algorithm/doc2vec.model'

model.save(model_path)

In [13]:
'''
    モデル内に保持されているベクトルの数を取得
    （ラベルを回答IDにすると、ラベルの数が戻る。
    　同一回答IDのサンプルのベクトルが、
    　全て上書きされていると考えられる）
'''
len(model.docvecs)

173

### (2-4) 予測処理

In [14]:
loaded_model = models.Doc2Vec.load(model_path)
loaded_model

In [15]:
len(loaded_model.docvecs)

173

In [16]:
'''
    予測処理にかけるコーパスを生成
'''
test_corpus1 = ['情報','システム','アドレス']
test_corpus2 = ['マウス','破損']

In [24]:
'''
    情報システムのアドレス（正解＝7040）
'''
inferred_vector1 = loaded_model.infer_vector(test_corpus1)
loaded_model.docvecs.most_similar([inferred_vector1])

[(7040, 0.6151561141014099),
 (4621, 0.6114732027053833),
 (7068, 0.5491945147514343),
 (4563, 0.5432988405227661),
 (4568, 0.5397956967353821),
 (4524, 0.5377195477485657),
 (4566, 0.5245414972305298),
 (4498, 0.5226050019264221),
 (4460, 0.516629695892334),
 (4602, 0.5153311491012573)]

In [18]:
'''
    マウス破損（正解＝4458）
'''
inferred_vector2 = loaded_model.infer_vector(test_corpus2)
loaded_model.docvecs.most_similar([inferred_vector2])

[(4458, 0.6497591733932495),
 (4459, 0.5751404166221619),
 (4530, 0.5314574241638184),
 (7065, 0.5107617378234863),
 (4562, 0.4856840968132019),
 (7064, 0.48553958535194397),
 (4554, 0.47325271368026733),
 (4423, 0.4729366600513458),
 (4455, 0.4703531265258789),
 (4598, 0.4658195376396179)]